In [96]:
import numpy as np
from numpy import pi,cos,sin
i = np.array([[1, 0], [0, 1]])
x = np.array([[0, 1], [1, 0]])
y = np.array([[0, -1j], [1j, 0]])
z = np.array([[1, 0], [0, -1]])
cx = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
ry = lambda theta: np.array([[cos(theta/2), -sin(theta/2)], [sin(theta/2), cos(theta/2)]])
rx = lambda theta: np.array([[cos(theta/2), -1j*sin(theta/2)], [-1j*sin(theta/2), cos(theta/2)]])
rz = lambda theta: np.array([[np.exp(-1j*theta/2), 0], [0, np.exp(1j*theta/2)]])

KeyError: 0

In [171]:
from sojo.stabilizer import PauliWord
from sojo.pc import PauliComposer
from sojo.tabular import stabilizer_tabular
import numpy as np
class PauliTerm:
    def __init__(self, words):
        self.words: dict[str, list[np.complex64]] = words
        self.num_qubits = len(next(iter(words)))
    def __str__(self):
        return ' + '.join([str(np.round(v, 4)) + '*' + str(k) for k,v in self.words.items()])
    def to_matrix_naive(self):
        # Return sum(P)
        matrix = np.zeros((2**(self.num_qubits), 2**(self.num_qubits)), dtype=complex)
        for word, scalar in self.words.items():
            matrix += PauliComposer(word, scalar).to_matrix()
        return matrix
    def reduce(self):
        # Example: P = [1*zxx, 1*yzi, 1j*zxx] -- reduce() --> [(1+1j)*zxx, 1*yzi]
        # Example: P = [1*zxx, 1*yzi, -1*zxx] -- reduce() --> [1*yzi]
        element_sum = {}
        for pauli_word in self.words:
            scalar, string = pauli_word.scalar, pauli_word.word
            if np.abs(scalar) < 10**(-10):
                scalar = 0
            if string in element_sum:
                element_sum[string] += scalar
            else:
                element_sum[string] = scalar

        self.words = [PauliWord(value, key) for key, value in element_sum.items() if value != 0]
        return
    def to_matrix(self, mode):
        # Return sum(P)
        if mode == 'csr':
            matrix = self.words[0].to_pc().to_csr()
            for word in self.words[1:]:
                matrix = matrix + word.to_pc().to_csr()
        else:
            matrix = self.words[0].to_pc().to_coo()
            for word in self.words[1:]:
                matrix = matrix + word.to_pc().to_coo()
        return matrix
    def multiply(self, other):
        # Multiply two PauliTerms
        new_terms = []
        for i in self.words:
            for j in other.words:
                new_terms.append(i.multiply(j))
        return PauliTerm(new_terms)
    def map(self, gate, index, param = 0):
        # Example: xii -- (h, 0) --> [z]ii
        # Example: xxx -- (t, 0) --> [x']xx = [1/sqrt(2) (x + y)]xx
        # Example: zxz -- (cx, [0,2]) --> [i]x[z]
        # This function will map a Pauli term to another Pauli term
        # Example: xii + xxx -- (h, 0) --> [z]ii + [z]xx
        # Two keys: only act on coressponding qubits, and independence between pauli strings
        num_words = len(self.words)
        count = 0
        def update_word(word, index, character):
            return word[:index] + character + word[index+1:]
        for word_j, scalar_j in list(self.words.items()):
            # Process on a single Pauli string
            if count > num_words:
                break
            count += 1
            
            if gate == 'cx':
                # Index will be [control, target]
                # Word will be [word_control, word_target]
                out_scalar, output_word = stabilizer_tabular(
                    word_j[index[0]] + word_j[index[1]], gate)
                # Replace word_j with output_word
                # Example: xii -- (cx, [0,2]) --> [x]i[x]
                new_word = update_word(word_j, index[0], output_word[0])
                new_word = update_word(new_word, index[1], output_word[1])
                # Don't forget +- 1 factor from cx
                # If new_word is not in the dictionary, add it, and set [0] in the old word = 0
                # If new_word is in the dictionary, update the scalar and append at [-1] in the existance one
                # New_word = word_j in cnot mean there is no change, either scalar
                if new_word == word_j:
                    pass
                else:
                    if new_word in self.words:
                        self.words[new_word].append(self.words[word_j][0] * out_scalar)
                        self.words[word_j][0] = 0
                    else:
                        self.words[new_word] = [self.words[word_j][0] * out_scalar]
                        self.words[word_j][0] = 0
                # if abs(self.words[new_word]) < 10**(-10):
                #     self.words.pop(new_word)
            else:
                # Index will be just a scalar
                if gate in ['rx', 'ry', 'rz']:
                    out_scalar, output_word = stabilizer_tabular(
                        word_j[index], gate, param)
                else:
                    out_scalar, output_word = stabilizer_tabular(
                        word_j[index], gate)
                if type(output_word) == list:
                    # One word turn to be two words, 
                    # Only one index, 2 output words, 2 output scalars
                    # I append new words to the end of the list
                    # Example: xii -- (ry, 0) --> [x]ii + [z]ii
                    new_word_0 = word_j
                    new_word_1 = update_word(word_j, index, output_word[1])
                    if new_word_1 in self.words:
                        self.words[new_word_1].append(self.words[word_j][0] * out_scalar[1])

                    else:
                        self.words[new_word_1] = [self.words[word_j][0] * out_scalar[1]]
                    self.words[word_j][0] *= out_scalar[0]
                else:
                    new_word = update_word(word_j, index, output_word)
                    if new_word in self.words:
                        self.words[new_word].append(self.words[word_j][0] * out_scalar)
                        self.words[word_j][0] = 0
                    else:
                        self.words[new_word] = [self.words[word_j][0] * out_scalar]
                        self.words[word_j][0] = 0
        self.words = {key: [s] for key, value in self.words.items() if abs(s := sum(value)) > 10**(-10)}
        return

In [151]:
k = {'xii': [0, 0, 1], 'yzi': [1, -1], 'zxx': [1]}

# Update value to be the sum of the list
k = {key: [s] for key, value in k.items() if (s := sum(value)) != 0}

print(k)


{'xii': [1], 'zxx': [1]}


In [172]:
k = PauliTerm({'zi': [1]})
# k.map('h', 0)
# k.map('cx', [0, 1])
# k.map('t', 0)
# k.map('cx', [0, 1])
# k.map('h', 0)

k.map('ry', 0, np.pi/3)
k.map('rx', 0, np.pi/2)
k.map('cx', [0,1])

k.map('t', 0)
k.map('cx', [0,1])
k.map('h', 0)
k.map('rz', 0, np.pi/4)
k.map('rx', 1, np.pi/3)
print(k)

[-0.183]*yi + [0.9659]*zi + [0.183]*xi


In [139]:
from collections import defaultdict

def reduce_weighted_sum(weighted_terms):
    term_dict = defaultdict(float)  # Dictionary to sum coefficients for each string
    for term in weighted_terms:
        coefficient, string = term  # Unpack the coefficient and string
        term_dict[string] += coefficient  # Sum coefficients for the same string

    # Construct the simplified result as a list of tuples
    reduced_terms = [(coef, string) for string, coef in term_dict.items() if coef != 0]

    return reduced_terms

# Example usage:
terms = [(0.5, 'xzz'), (0.7, 'yiii'), (-0.5, 'xzz')]
reduced = reduce_weighted_sum(terms)
print(reduced)  # [(0.6, 'xzz'), (0.7, 'yiii')]


[(0.7, 'yiii')]
